In [116]:
import math
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.base import clone
from google.colab import drive

from sklearn import ensemble
from sklearn import neighbors

import re
import string
import spacy

In [44]:
# utils

drive.mount('/content/drive')
PATH = "/content/drive/MyDrive/Colab Notebooks/ML lab 06/Лабораторные/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
df = pd.read_csv(PATH + "SMS.tsv", sep='\t')

df

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [46]:
X = df.text
y = df['class']

In [47]:
# X, X_rem, y, y_rem = train_test_split(X, y, test_size=0.95)

In [48]:
X.shape

(278,)

In [49]:
text = X.values
sp = spacy.load('en_core_web_sm')

for i in range(len(text)):
  text[i] = ' '.join([word.lemma_ for word in sp(str(text[i]))])

stopwords = sp.Defaults.stop_words
text = np.array([' '.join([word if word not in stopwords else '' for word in post.split()]) for post in text])

text = np.array(list(map(lambda x: x.lower(), text)))
text = np.array(list(map(lambda x: re.sub('\n+', ' ', x), text)))
text = np.array(list(map(lambda x: re.sub('\d+', ' ', x), text)))
text = np.array(list(map(lambda x: x.translate(str.maketrans('', '', string.punctuation)), text)))
text = np.array(list(map(lambda x: re.sub('[^a-zA-Z ]', '', x), text)))
text = np.array(list(map(lambda x: re.sub(' +', ' ', x), text)))
text = np.array(list(map(lambda x: x.strip(), text)))



X = text

In [50]:
vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(X)

X = pd.DataFrame(vectorized.toarray(), columns=vectorizer.get_feature_names_out())
X

,aathi,abiola,able,abt,accept,accordingly,account,actually,ad,address,...,yesterday,yi,yijue,ym,yo,youcouk,youi,yr,yrs,yup
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.216981,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
X.shape

(278, 1018)

In [52]:
y = (y == 'spam').astype(int)

In [83]:
def entropy(y):
    """Calculate entropy of a target variable"""
    _, counts = np.unique(y, return_counts=True)
    probs = counts / len(y)
    return -np.sum(probs * np.log2(probs))

def information_gain(X, y, feature_idx):
    """Calculate information gain of a feature"""
    H_total = entropy(y)
    _, counts = np.unique(X[:, feature_idx], return_counts=True)
    probs = counts / len(X)
    H_feature = np.sum(probs * entropy(y[X[:, feature_idx] == 1]))
    return H_total - H_feature

def feature_filtration(X, y, k=30):
    """Perform feature filtration with information gain metric"""
    scores = []
    for i in range(X.shape[1]):
        scores.append((i, information_gain(X, y, i)))
    scores.sort(key=lambda x: x[1], reverse=True)
    top_k_features = [x[0] for x in scores[:k]]
    return top_k_features

In [84]:
def sequential_feature_selection(X, y, clf, k = 30):
  """SFS wrapper method"""
  best_features = []
  features = set(range(X.shape[1]))

  while len(best_features) < k:
    scores = []
    for i in features:
      if i in best_features:
        continue
      new_x = X[:, best_features + [i]]
      x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
      
      clf.fit(x_train, y_train)
      scores.append(f1_score(clf.predict(x_test), y_test))

    max_s = np.argsort(scores)[-1]

    best_features.append(max_s)

  return best_features

In [85]:
def recursive_feature_elimination(X, y, k = 988):
    k = X.shape[1] - k
    best_features = list(range(X.shape[1]))
    while len(best_features) > k:
        model = LogisticRegression()
        model.fit(X[:, best_features], y)
        best_features = np.argsort(np.abs(model.coef_.ravel()))[1:]
    return best_features

In [ ]:
X_filtered = feature_filtration(X.values, y)

X_filtered

In [ ]:
lr = LogisticRegression()
X_sfs =sequential_feature_selection(X.values, y, lr)

X_sfs

In [ ]:
'''
[862,
 756,
 740,
 560,
 77,
 468,
 44,
 248,
 548,
 509,
 839,
 39,
 251,
 524,
 440,
 829,
 760,
 265,
 214,
 791,
 444,
 982,
 686,
 808,
 707,
 156,
 606,
 31,
 735,
 513]
'''

In [88]:
X_rfe = recursive_feature_elimination(X.values, y)

X_rfe

array([28, 27, 26, 25, 24, 23, 22, 20, 19, 17, 16, 13, 15,  7,  2, 11, 10,
        9,  8,  4, 12,  6,  5,  3, 14, 18,  1, 21, 30, 29])

In [ ]:
'''
[28, 27, 26, 25, 24, 23, 22, 20, 19, 17, 16, 13, 15,  7,  2, 11, 10,
        9,  8,  4, 12,  6,  5,  3, 14, 18,  1, 21, 30, 29]
'''

In [91]:
print([X.columns[i] for i in X_filtered])

['aathi', 'abiola', 'able', 'abt', 'accept', 'accordingly', 'account', 'actually', 'ad', 'address', 'admirer', 'adsense', 'advice', 'advise', 'aeronautics', 'aeroplane', 'afraid', 'aft', 'afternoon', 'aftr', 'age', 'ages', 'ah', 'aight', 'allow', 'alright', 'alternativehope', 'alwys', 'am', 'ambrith']


In [92]:
print([X.columns[i] for i in X_sfs])

['thank', 'season', 'sad', 'money', 'billing', 'lady', 'arrange', 'dt', 'miwa', 'ltgt', 'suganya', 'april', 'dunno', 'marrge', 'itz', 'student', 'self', 'entry', 'denis', 'skip', 'jay', 'won', 'professor', 'space', 'reason', 'comb', 'ntt', 'anna', 'row', 'luv']


In [93]:
print([X.columns[i] for i in X_rfe])

['am', 'alwys', 'alternativehope', 'alright', 'allow', 'aight', 'ah', 'age', 'aftr', 'aft', 'afraid', 'advise', 'aeroplane', 'actually', 'able', 'adsense', 'admirer', 'address', 'ad', 'accept', 'advice', 'account', 'accordingly', 'abt', 'aeronautics', 'afternoon', 'abiola', 'ages', 'amp', 'ambrith']


In [100]:
from sklearn.feature_selection import SelectKBest, SelectFromModel, SelectFpr, chi2

In [122]:
selector = SelectKBest(chi2, k=30)
selector.fit(X.values, y)
X_kbest = selector.get_support(indices=True)
print([X.columns[i] for i in X_kbest])

['apply', 'award', 'bored', 'box', 'claim', 'contact', 'customer', 'date', 'detail', 'entry', 'free', 'guarantee', 'housewife', 'ipod', 'min', 'mobile', 'national', 'po', 'pobox', 'ppm', 'prize', 'rate', 'ringtone', 'service', 'tone', 'txt', 'video', 'weekly', 'win', 'won']


In [99]:
selector = SelectFromModel(SVC(kernel='linear'), max_features=30)
selector.fit(X, y)
X_from_model = selector.get_support(indices=True)
print([X.columns[i] for i in X_from_model])

['apply', 'award', 'box', 'claim', 'contact', 'customer', 'date', 'entry', 'free', 'ls', 'ltgt', 'min', 'mobile', 'national', 'new', 'ok', 'po', 'pobox', 'ppm', 'prize', 'rate', 'ringtone', 'service', 'stop', 'tone', 'txt', 'value', 'video', 'wap', 'win']


In [102]:
selector = SelectFpr(chi2)
selector.fit(X.values, y)
X_fpr = selector.get_support(indices=True)[:30]
print([X.columns[i] for i in X_fpr])

['apply', 'award', 'bored', 'box', 'charge', 'claim', 'contact', 'customer', 'date', 'detail', 'draw', 'entry', 'free', 'guarantee', 'housewife', 'ipod', 'ls', 'min', 'mobile', 'national', 'new', 'po', 'pobox', 'ppm', 'prize', 'rate', 'representative', 'ringtone', 'service', 'stop']


In [118]:
X_train = pd.DataFrame(X, columns=[X.columns[i] for i in X_filtered])
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2)
rf_cls = ensemble.RandomForestClassifier()
knn_cls = neighbors.KNeighborsClassifier()
lg_cls =LogisticRegression()
svc_cls =SVC()
rf_cls.fit(X_train, y_train)
knn_cls.fit(X_train, y_train)
lg_cls.fit(X_train, y_train)
svc_cls.fit(X_train, y_train)

pred = rf_cls.predict(X_test)
print("RF: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = knn_cls.predict(X_test)
print("KNN: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = lg_cls.predict(X_test)
print("LG: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred =  svc_cls.predict(X_test)
print("SVC: accuracy_score: " + str(accuracy_score(pred, y_test)))

RF: accuracy_score: 0.8928571428571429
KNN: accuracy_score: 0.8928571428571429
LG: accuracy_score: 0.8928571428571429
SVC: accuracy_score: 0.8928571428571429


In [119]:
X_train = pd.DataFrame(X, columns=[X.columns[i] for i in X_sfs])
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2)
rf_cls = ensemble.RandomForestClassifier()
knn_cls = neighbors.KNeighborsClassifier()
lg_cls =LogisticRegression()
svc_cls =SVC()
rf_cls.fit(X_train, y_train)
knn_cls.fit(X_train, y_train)
lg_cls.fit(X_train, y_train)
svc_cls.fit(X_train, y_train)

pred = rf_cls.predict(X_test)
print("RF: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = knn_cls.predict(X_test)
print("KNN: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = lg_cls.predict(X_test)
print("LG: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred =  svc_cls.predict(X_test)
print("SVC: accuracy_score: " + str(accuracy_score(pred, y_test)))

RF: accuracy_score: 0.8928571428571429
KNN: accuracy_score: 0.8571428571428571
LG: accuracy_score: 0.8571428571428571
SVC: accuracy_score: 0.875


In [120]:
X_train = pd.DataFrame(X, columns=[X.columns[i] for i in X_rfe])
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2)
rf_cls = ensemble.RandomForestClassifier()
knn_cls = neighbors.KNeighborsClassifier()
lg_cls =LogisticRegression()
svc_cls =SVC()
rf_cls.fit(X_train, y_train)
knn_cls.fit(X_train, y_train)
lg_cls.fit(X_train, y_train)
svc_cls.fit(X_train, y_train)

pred = rf_cls.predict(X_test)
print("RF: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = knn_cls.predict(X_test)
print("KNN: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = lg_cls.predict(X_test)
print("LG: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred =  svc_cls.predict(X_test)
print("SVC: accuracy_score: " + str(accuracy_score(pred, y_test)))

RF: accuracy_score: 0.8571428571428571
KNN: accuracy_score: 0.8571428571428571
LG: accuracy_score: 0.8571428571428571
SVC: accuracy_score: 0.8571428571428571


In [123]:
X_train = pd.DataFrame(X, columns=[X.columns[i] for i in X_kbest])
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2)
rf_cls = ensemble.RandomForestClassifier()
knn_cls = neighbors.KNeighborsClassifier()
lg_cls =LogisticRegression()
svc_cls =SVC()
rf_cls.fit(X_train, y_train)
knn_cls.fit(X_train, y_train)
lg_cls.fit(X_train, y_train)
svc_cls.fit(X_train, y_train)

pred = rf_cls.predict(X_test)
print("RF: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = knn_cls.predict(X_test)
print("KNN: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = lg_cls.predict(X_test)
print("LG: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred =  svc_cls.predict(X_test)
print("SVC: accuracy_score: " + str(accuracy_score(pred, y_test)))

RF: accuracy_score: 0.9821428571428571
KNN: accuracy_score: 0.9464285714285714
LG: accuracy_score: 0.8928571428571429
SVC: accuracy_score: 0.9642857142857143


In [124]:
X_train = pd.DataFrame(X, columns=[X.columns[i] for i in X_from_model])
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2)
rf_cls = ensemble.RandomForestClassifier()
knn_cls = neighbors.KNeighborsClassifier()
lg_cls =LogisticRegression()
svc_cls =SVC()
rf_cls.fit(X_train, y_train)
knn_cls.fit(X_train, y_train)
lg_cls.fit(X_train, y_train)
svc_cls.fit(X_train, y_train)

pred = rf_cls.predict(X_test)
print("RF: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = knn_cls.predict(X_test)
print("KNN: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = lg_cls.predict(X_test)
print("LG: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred =  svc_cls.predict(X_test)
print("SVC: accuracy_score: " + str(accuracy_score(pred, y_test)))

RF: accuracy_score: 1.0
KNN: accuracy_score: 0.9642857142857143
LG: accuracy_score: 0.9285714285714286
SVC: accuracy_score: 0.9821428571428571


In [125]:
X_train = pd.DataFrame(X, columns=[X.columns[i] for i in X_fpr])
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2)
rf_cls = ensemble.RandomForestClassifier()
knn_cls = neighbors.KNeighborsClassifier()
lg_cls =LogisticRegression()
svc_cls =SVC()
rf_cls.fit(X_train, y_train)
knn_cls.fit(X_train, y_train)
lg_cls.fit(X_train, y_train)
svc_cls.fit(X_train, y_train)

pred = rf_cls.predict(X_test)
print("RF: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = knn_cls.predict(X_test)
print("KNN: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred = lg_cls.predict(X_test)
print("LG: accuracy_score: " + str(accuracy_score(pred, y_test)))

pred =  svc_cls.predict(X_test)
print("SVC: accuracy_score: " + str(accuracy_score(pred, y_test)))

RF: accuracy_score: 0.9464285714285714
KNN: accuracy_score: 0.8928571428571429
LG: accuracy_score: 0.8571428571428571
SVC: accuracy_score: 0.9821428571428571
